<img src="./images/DLI_Header.png" style="width: 400px;">



# 3.0 Multi-Node Distributed Training Strategies

In this notebook, we will learn how to run [NeMo](https://github.com/NVIDIA/NeMo/) GPT pretraing on multiple nodes.


## The goals

The goals of this notebook are to:
* Run simple multi-node training of NeMo Framework scripts
* Run a hybrid multi-node execution with data, tensor and pipeline parallel distributions


**[3.1 Multi-Node Training Execution of NeMo GPT Pretraining](#3.1-Multi-Node-Training-Execution-of-NeMo-GPT-Pretraining)<br>**
**[3.2 Multi-Node Execution with Data Parallelism](#3.2-Multi-Node-Execution-with-Data-Parallelism)<br>**
**[3.3 Inter/Intra Node Communications](#3.3-Inter/Intra-Node-Communications)<br>**
**[3.4 Monitoring and Profiling the Training](#3.4-Monitoring-and-Profiling-the-Training)<br>**
**[3.5 Increase The Batchsize / GPU](#3.5-Increase-The-Batchsize-/-GPU)<br>**
**[3.6 Exercise: Hybrid Distributed Training Strategy](#3.6-Exercise:-Hybrid-Distributed-Training-Strategy)<br>**

### Cancel Previous Running/Pending Jobs

Before moving on, check that no jobs are still running or waiting on the SLURM queue. Let's check the SLURM jobs queue by executing the following cell:

In [1]:
# Check the SLURM jobs queue 
!squeue

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


If there are still jobs running or pending, execute the following cell to cancel all the user's jobs using the `scancel` command. 

In [2]:
# Cancel admin user jobs
!scancel -u $USER

# Check again the SLURM jobs queue (should be either empty, or the status TS column should be CG)
!squeue

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


---
# 3.1 Multi-Node Training Execution of NeMo GPT Pretraining

In the previous notebook, we submitted our jobs in an interactive session after allocating 1 node. 

For multi-node jobs, we need to rely on the SLURM scheduler. By default, multi-node training with NeMo Framework uses the [NVIDIA Collective Communications Library - NCCL](https://developer.nvidia.com/nccl) distributed backend of the [PyTorch distributed launcher](https://pytorch.org/docs/stable/elastic/run.html).

For 2-Node execution, we will use `SBATCH` scripting. While Python execution commands and their arguments remain similar to how it was done for single node, we will need additional `SBATCH` arguments for resource allocation. 

Let's start by executing NeMo GPT pretraining on 2 nodes using only Data Parallelism, meaning that the model is copied on the 4 allocated GPUs, each processing different data batches.

# 3.2 Multi-Node Execution with Data Parallelism

In the previous 2-GPU data parallelism execution, the batch size processed by each GPU was 16 (set by `model.micro_batch_size`) corresponding to a global batch size of 32 (set by `model.global_batch_size`). 

When using 4 GPUs, we can keep the micro batch size per GPUs being 16 and set the global batch size to 64 (`MICRO_BATCH_SIZE` $ \times $ 4).

Let's have a look at the script before allocating resources and executing it. 

Notice the `SBATCH` arguments for allocating resources. `#SBATCH --ntasks-per-node` should be set to 2 when using NeMo Framework as we will use both GPUs on every node.

In [3]:
# Have a look at NeMo GPT pretraining execution on 2 nodes
!cat /dli/code/pretrain_gpt_2Node4GPU.sh

#!/bin/bash
#SBATCH --job-name=dli_2nodes
#SBATCH --nodes=2
#SBATCH --ntasks-per-node=2       
#SBATCH --cpus-per-task=32 ### Number of threads per task (OMP threads)
#SBATCH -o /dli/nemo/logs/%j.out
#SBATCH -e /dli/nemo/logs/%j.err

set -x -e

export OMP_NUM_THREADS=$SLURM_CPUS_PER_TASK

# Distributed training args
NNODES=2
GPUS_PER_NODE=2
TP_SIZE=1
PP_SIZE=1 

# Distributed training 
MICRO_BATCH_SIZE=16
GLOBAL_BATCH_SIZE=64    # <--- CHANGED HERE

# Model architecture 
NLAYERS=12
NHIDDEN=768
NHEADS=32
SEQ_LEN=1024

# Data Paths
VOCAB_FILE=/dli/data/GPT-2_assets/gpt2-vocab.json
MERGE_FILE=/dli/data/GPT-2_assets/gpt2-merges.txt
DATA_PATH=[1.0,/dli/data/GPT-2_assets/my-gpt2_text_document]

OUTPUT_PATH=/dli/nemo
LOGS_PATH=/dli/nemo/logs
NAME="2Nodes4GPUS"       # <--- CHANGED HERE 


OPTIMIZER_ARGS=" \
            model.optim.name=fused_adam \
            model.optim.betas=[0.9,0.95] \
            model.optim.lr=6e-5 \
            model.optim.sched.min_lr=6e-6 \
            model.optim.s

Now, let's submit the SLURM job using `sbatch` [`pretrain_gpt_2Node4GPU.sh`](./code/pretrain_gpt_2Node4GPU.sh) and check the SLURM queue using the `squeue` command.

In [4]:
# Submit the 2 nodes jobs
!sbatch /dli/code/pretrain_gpt_2Node4GPU.sh

# Check the SLURM queue
!squeue

Submitted batch job 12
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
                12  slurmpar dli_2nod     root PD       0:00      2 (None)


We can check the GPU usage by running the `nvidia-smi` command on the master lab node. After a few seconds, when the nodes are allocated and the script begins its execution, we should see the GPUs 0,1,2,3 utilized, as shown below. Please notice that if it's necessary to wait until the actual training starts. Until then, you will not be able to see any GPU activity.


<img src="images/2N_4gpus_utilization.png" width="650"/>

In [5]:
# Check GPU utilization on the master node
!sleep 60
!nvidia-smi

Thu Mar 21 21:33:07 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.12             Driver Version: 535.104.12   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          On  | 00000001:00:00.0 Off |                    0 |
| N/A   53C    P0             310W / 300W |  79551MiB / 81920MiB |     81%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

To understand the performance of the NeMo GPT-3 pretraining, we can check the generated [logs](./nemo/logs/2Nodes4GPUS.txt) during execution.

Let's first verify the world size of our run. We should see this:
```
Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/4
Initializing distributed: GLOBAL_RANK: 1, MEMBER: 2/4
Initializing distributed: GLOBAL_RANK: 2, MEMBER: 3/4
Initializing distributed: GLOBAL_RANK: 3, MEMBER: 4/4
```

In [6]:
!grep "Initializing distributed:" /dli/nemo/logs/2Nodes4GPUS.txt

Initializing distributed: GLOBAL_RANK: 1, MEMBER: 2/4
Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/4
Initializing distributed: GLOBAL_RANK: 2, MEMBER: 3/4
Initializing distributed: GLOBAL_RANK: 3, MEMBER: 4/4


Let's now check the performance of the GPT pretraining on 4 GPUs.

In [7]:
!cat /dli/nemo/logs/2Nodes4GPUS.txt | grep Epoch | tail -4

Epoch 0:  20%|██        | 30/150 [00:35<02:20,  1.17s/it, loss=10.5, v_num=, reduced_train_loss=10.30, global_step=19.00, consumed_samples=1216.0, val_loss=10.20]
Epoch 0, global step 20: 'val_loss' reached 10.20249 (best 10.20249), saving model to '/dli/nemo/2Nodes4GPUS/checkpoints/megatron_gpt--val_loss=10.20-step=20-consumed_samples=1216.0.ckpt' as top 10


From the extract logs, we can see the training performance while using 4 GPUs (2 GPUs per node). 

```
Epoch 0: 100%|██████████| 150/150 [02:14<00:00,  1.12it/s, loss=8.43, v_num=, reduced_train_loss=8.370, global_step=99.00, consumed_samples=6336.0, val_loss=8.360]
```
Compare this with 1 node executions, and discuss it with the instructor.

---
# 3.3 Inter/Intra Node Communications 

In the previous run, you may have noticed that we added the NCCL variable `NCCL_DEBUG=INFO` in order to output the NCCL debug log traces during training as shown bellow. 
```
slurmnode1:2326:2509 [1] NCCL INFO Channel 00/0 : 1[200000] -> 2[300000] [send] via NET/Socket/0
slurmnode1:2326:2509 [1] NCCL INFO Channel 01/0 : 1[200000] -> 2[300000] [send] via NET/Socket/0
slurmnode1:2326:2509 [1] NCCL INFO Channel 00 : 1[200000] -> 0[100000] via SHM/direct/direct
slurmnode1:2326:2509 [1] NCCL INFO Channel 01 : 1[200000] -> 0[100000] via SHM/direct/direct
```
Let's check what NCCL reported in the generated [logs](./nemo/logs/2Nodes4GPUS.txt) during the previous execution.

In [8]:
!grep Channel /dli/nemo/logs/2Nodes4GPUS.txt | grep slurmnode1 | head

slurmnode1:3419:3604 [1] NCCL INFO Channel 00/0 : 1[200000] -> 2[300000] [send] via NET/Socket/0
slurmnode1:3419:3604 [1] NCCL INFO Channel 01/0 : 1[200000] -> 2[300000] [send] via NET/Socket/0
slurmnode1:3419:3604 [1] NCCL INFO Channel 00/0 : 1[200000] -> 0[100000] via P2P/IPC/read
slurmnode1:3419:3604 [1] NCCL INFO Channel 01/0 : 1[200000] -> 0[100000] via P2P/IPC/read
slurmnode1:3418:3602 [0] NCCL INFO Channel 00/02 :    0   1   2   3
slurmnode1:3418:3602 [0] NCCL INFO Channel 01/02 :    0   1   2   3
slurmnode1:3418:3602 [0] NCCL INFO Channel 00/0 : 3[400000] -> 0[100000] [receive] via NET/Socket/0
slurmnode1:3418:3602 [0] NCCL INFO Channel 01/0 : 3[400000] -> 0[100000] [receive] via NET/Socket/0
slurmnode1:3418:3602 [0] NCCL INFO Channel 00/0 : 0[100000] -> 1[200000] via P2P/IPC/read
slurmnode1:3418:3602 [0] NCCL INFO Channel 01/0 : 0[100000] -> 1[200000] via P2P/IPC/read


This will allow us to check the type of networking used between GPUs and nodes during the training. 

The internode communications are reported as `NET/Socket/0` while direct GPU-to-GPU communications are reported as `P2P/IPC`.
In our configuration, direct GPU-to-GPU communication should be used within nodes: GPU0<->GPU1 and GPU2<->GPU3). 

In [9]:
!grep Channel /dli/nemo/logs/2Nodes4GPUS.txt | grep P2P/IPC | head

slurmnode1:3419:3604 [1] NCCL INFO Channel 00/0 : 1[200000] -> 0[100000] via P2P/IPC/read
slurmnode1:3419:3604 [1] NCCL INFO Channel 01/0 : 1[200000] -> 0[100000] via P2P/IPC/read
slurmnode2:160:342 [1] NCCL INFO Channel 00/0 : 3[400000] -> 2[300000] via P2P/IPC/read
slurmnode2:160:342 [1] NCCL INFO Channel 01/0 : 3[400000] -> 2[300000] via P2P/IPC/read
slurmnode2:159:344 [0] NCCL INFO Channel 00/0 : 2[300000] -> 3[400000] via P2P/IPC/read
slurmnode2:159:344 [0] NCCL INFO Channel 01/0 : 2[300000] -> 3[400000] via P2P/IPC/read
slurmnode1:3418:3602 [0] NCCL INFO Channel 00/0 : 0[100000] -> 1[200000] via P2P/IPC/read
slurmnode1:3418:3602 [0] NCCL INFO Channel 01/0 : 0[100000] -> 1[200000] via P2P/IPC/read
slurmnode2:159:358 [0] NCCL INFO Channel 00/0 : 2[300000] -> 3[400000] via P2P/IPC/read
slurmnode2:159:358 [0] NCCL INFO Channel 01/0 : 2[300000] -> 3[400000] via P2P/IPC/read


---
# 3.4 Monitoring and Profiling Training


So far, monitoring the training runs with NeMo Framework was done via the text log files. However, monitoring training is also possible through the tensorboard visualization of the hyperparameters and training/evaluation metrics. In addition, visualizing can be helpful for debugging and optimizing the models during training.

## 3.4.1 Training Metrics Visualization on Tensorboard

<img src="images/tensorboard1.png" width="1024"/>

In the previous NeMo Framework runs, we set the arguments for recording Tensorboard events. The graphs of all the previous experiments are available in the folder `nemo`.

Note that to profile memory usage it's necessary to add a profiler to the PyTorch Lightning trainer, used internally by NeMo Framework. This can be done by adding the following lines to `megatron_gpt_pretraining.py`:
```
    if cfg.trainer.get('use_profiler', False):
        schedule = torch.profiler.schedule(wait=50, warmup=1, active=2, repeat=1)
        profiler = PyTorchProfiler(activities=[
                torch.profiler.ProfilerActivity.CPU,
                torch.profiler.ProfilerActivity.CUDA,
            ], 
            schedule=schedule, record_shapes=True, profile_memory=True, with_stack=True, with_flops=True, with_modules=True)
        del cfg.trainer.use_profiler
        trainer = Trainer(plugins=plugins, strategy=strategy, profiler=profiler, **cfg.trainer)
    else:
        trainer = Trainer(plugins=plugins, strategy=strategy, **cfg.trainer)
```

You can also add profiler via default option `+trainer.profiler=pytorch` (or `+trainer.profiler=simple`, `trainer.profiler=advanced`), but it will output less information.

Execute the next cell to create a link to Tensorboard for the browser. Then, click the link to see graphs of experiment metrics saved in the specified directory with `tensorboard` logs. 

In [10]:
%%js
const href = window.location.hostname +'/tensorboard/';
let a = document.createElement('a');
let link = document.createTextNode('Open Tensorboard!');
a.appendChild(link);
a.href = "http://" + href;
a.style.color = "navy"
a.target = "_blank"
element.append(a);

<IPython.core.display.Javascript object>

## 3.4.2 Pytorch Profiler with Tensorboard

Several existing profiling tools can be used to investigate bottlenecks during the training and inference processes. This allows us to identify the most expensive operations, issues such as GPU starvation, or unnecessary operations. 

In this class, we will use the [Pytorch Lightning Profiler](https://pytorch-lightning.readthedocs.io/en/1.5.10/advanced/profiler.html), a tool collecting performance metrics during Pytorch executions. The [Tensorboard-plugin](https://pytorch.org/tutorials/intermediate/tensorboard_profiler_tutorial.html) with the PyTorch Profiler provides a visualization of each GPU process, with several measures of operations running on each GPU, whether using hardware accelerator (TensorCores). It also provides recommendations to improve the process.

We will trace only the execution of the GPU_0.

For the previous run, profiling is available on the Tensorboard link at the `PYTORCH_PROFILER` tab.

<img src="images/profiling1.png" width="1280"/>

In case you already closed the Tensorboard page, you can re-generate the link by executing the next cell. Click the link to open Tensorboard and then, go to the `PYTORCH_PROFILER` tab.


In [11]:
%%js
const href = window.location.hostname +'/tensorboard/';
let a = document.createElement('a');
let link = document.createTextNode('Open Tensorboard!');
a.appendChild(link);
a.href = "http://" + href;
a.style.color = "navy"
a.target = "_blank"
element.append(a);

<IPython.core.display.Javascript object>

The profiler homepage shows an overview of tracing. 

### The Memory View
Let's have a look at the Memory View showing the memory allocation and deallocation of our run. We can zoom into the memory curve to see the related memory events.

<img src="images/profiling4.png" width="1280"/>

The memory allocation followed by deallocation corresponds to the forward and backward pass. This process is traced twice as we traced 2 training steps. We can also see that a peak of ~73GB of the GPU device memory is used during the training step.


Great!\
Before moving on, let's release some disk space by deleting the unnecessary checkpoints generated by the previous execution.

In [12]:
# Clean the checkpoints
!rm -rf /dli/nemo/2Nodes4GPUS/checkpoints 

---
# 3.5 Increase the Batchsize / GPU

**Special Warning:** In this section, Out Of Memory (OOM) issues are expected! No problem, we will see a few solutions addressing this problem in the next notebook.


The size of the current GPT model fits into the GPU memory and thus does not require model distribution (tensor or pipeline parallelism). \
Using only data parallelism, we can improve the training throughput (number of sequences processed per second) by increasing the batch size processed per GPU until reaching the maximum batch size that fits on GPU memory.\
Let's check this by increasing the data processed by each GPU. When using 4 GPUs with only data parallelism (DP_SIZE $= 4$), by increasing the micro batch size per GPUs from 16 to 32, the global batch size should be MICRO_BATCH_SIZE $\times$ DP_SIZE $= 128$.

Let's prepare the training script for this scenario by running the following cell.

In [13]:
%%writefile /dli/code/pretrain_gpt_2Node4GPU_DP_4_MBS_32.sh
#!/bin/bash
#SBATCH --job-name=dli_2nodes
#SBATCH --nodes=2
#SBATCH --ntasks-per-node=2       
#SBATCH --cpus-per-task=32 ### Number of threads per task (OMP threads)
#SBATCH -o /dli/nemo/logs/%j.out
#SBATCH -e /dli/nemo/logs/%j.err

set -x -e

export OMP_NUM_THREADS=$SLURM_CPUS_PER_TASK

# Distributed training args
NNODES=2
GPUS_PER_NODE=2
TP_SIZE=1
PP_SIZE=1 

# Distributed training 
MICRO_BATCH_SIZE=32      # <--- CHANGED HERE
GLOBAL_BATCH_SIZE=128    # <--- CHANGED HERE

# Model architecture 
NLAYERS=12
NHIDDEN=768
NHEADS=32
SEQ_LEN=1024

# Data Paths
VOCAB_FILE=/dli/data/GPT-2_assets/gpt2-vocab.json
MERGE_FILE=/dli/data/GPT-2_assets/gpt2-merges.txt
DATA_PATH=[1.0,/dli/data/GPT-2_assets/my-gpt2_text_document]

OUTPUT_PATH=/dli/nemo
LOGS_PATH=/dli/nemo/logs
NAME="2Nodes4GPUS_DP_4_MBS_32"


OPTIMIZER_ARGS=" \
            model.optim.name=fused_adam \
            model.optim.betas=[0.9,0.95] \
            model.optim.lr=6e-5 \
            model.optim.sched.min_lr=6e-6 \
            model.optim.sched.name=CosineAnnealing \
            +model.optim.sched.max_steps=800 \
            model.optim.sched.warmup_steps=80 \
            model.optim.weight_decay=1e-1 \
        "

TRAINER_ARGS=" \
            trainer.gradient_clip_val=1.0 \
            trainer.precision=32 \
            trainer.devices=$GPUS_PER_NODE \
            trainer.num_nodes=$NNODES \
            trainer.max_steps=100 \
            trainer.enable_model_summary=true \
            trainer.log_every_n_steps=10 \
            trainer.val_check_interval=20 \
            trainer.limit_val_batches=10 \
            +trainer.use_profiler=true \
        "

GPT_ARGS=" \
            model.num_layers=$NLAYERS \
            model.hidden_size=$NHIDDEN \
            model.num_attention_heads=$NHEADS \
            model.encoder_seq_length=$SEQ_LEN \
            model.data.seq_length=$SEQ_LEN \
            model.max_position_embeddings=$SEQ_LEN \
            model.micro_batch_size=$MICRO_BATCH_SIZE \
            model.global_batch_size=$GLOBAL_BATCH_SIZE \
            model.tokenizer.vocab_file=$VOCAB_FILE \
            model.tokenizer.merge_file=$MERGE_FILE \
            model.init_method_std=0.006 \
            $OPTIMIZER_ARGS \
        "

OUTPUT_ARGS=" \
            exp_manager.explicit_log_dir=$OUTPUT_PATH/$NAME \
            exp_manager.resume_if_exists=false \
            exp_manager.name=$NAME \
        "

PARALLEL_ARGS=" \
            model.tensor_model_parallel_size=$TP_SIZE \
            model.pipeline_model_parallel_size=$PP_SIZE \
        "

export CMD=" \
            python /dli/code/NeMo/examples/nlp/language_modeling/megatron_gpt_pretraining.py \
            --config-path=/dli/code/NeMo/examples/nlp/language_modeling/conf/ \
            --config-name=megatron_gpt_config.yaml \
            $TRAINER_ARGS \
            $PARALLEL_ARGS \
            $GPT_ARGS \
            $OUTPUT_ARGS \
            model.data.data_prefix=$DATA_PATH \
            model.data.data_impl=mmap \
            model.data.splits_string=\"949,50,1\" \
        "

clear; srun --jobid $SLURM_JOBID bash -c 'NCCL_DEBUG=INFO $CMD' 2>&1 | tee -a $LOGS_PATH/$NAME.txt

Overwriting /dli/code/pretrain_gpt_2Node4GPU_DP_4_MBS_32.sh


Now, let's submit the previous sbatch script [pretrain_gpt_2Node4GPU_DP_4_MBS_32.sh](./code/pretrain_gpt_2Node4GPU_DP_4_MBS_32.sh) and check the SLURM queue using the squeue command.

In [14]:
# submit the 2 nodes jobs
!sbatch /dli/code/pretrain_gpt_2Node4GPU_DP_4_MBS_32.sh

# Check the SLURM queue
!squeue

Submitted batch job 13
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
                13  slurmpar dli_2nod     root  R       0:00      2 slurmnode[1-2]


Let's now check the NeMo GPT-3 pretraining performance by looking at the generated [logs](./nemo/logs/2Nodes4GPUS_DP_4_MBS_32.txt) during the execution.

In [15]:
!sleep 60
!grep Epoch /dli/nemo/logs/2Nodes4GPUS_DP_4_MBS_32.txt

Epoch 0:   0%|          | 0/150 [00:17<?, ?it/s]slurmnode1:5489:5684 [0] NCCL INFO [Service thread] Connection closed by localRank 0



No elapsed time per iteration is shown. **What just happened?!** 

Are we saturating the GPU memory? Run the cell bellow to search for RuntimeError in our logs. You should see:
```
torch.cuda.OutOfMemoryError: CUDA out of memory. Tried to allocate 4.00 GiB (GPU 1; 79.35 GiB total capacity; 70.19 GiB already allocated; 3.61 GiB free; 74.24 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF
torch.cuda.OutOfMemoryError: CUDA out of memory. Tried to allocate 4.00 GiB (GPU 0; 79.35 GiB total capacity; 70.19 GiB already allocated; 3.61 GiB free; 74.24 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF
```

In [17]:
!grep "torch.cuda.OutOfMemoryError" /dli/nemo/logs/2Nodes4GPUS_DP_4_MBS_32.txt

torch.cuda.OutOfMemoryError: CUDA out of memory. Tried to allocate 4.00 GiB (GPU 1; 79.15 GiB total capacity; 76.99 GiB already allocated; 135.25 MiB free; 78.21 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF
torch.cuda.OutOfMemoryError: CUDA out of memory. Tried to allocate 4.00 GiB (GPU 0; 79.15 GiB total capacity; 76.99 GiB already allocated; 135.25 MiB free; 78.21 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF
torch.cuda.OutOfMemoryError: CUDA out of memory. Tried to allocate 4.00 GiB (GPU 0; 79.15 GiB total capacity; 76.99 GiB already allocated; 135.25 MiB free; 78.21 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid f


Indeed, we should see `CUDA out of memory` errors which means that the GPU memory cannot handle training this transformer model size with the specified arguments.

Doubling the amount of data processed per GPU is too much for the 80Gb of memory.  

In the next lab, we will focus on how to address this issue by reducing the model's memory footprint.

Great, before moving on, let's release some disk space by deleting the unnecessary checkpoints generated by the previous execution.

In [18]:
# Clean the checkpoints and logs directory
!rm -rf /dli/nemo/2Nodes4GPUS_DP_4_MBS_32/checkpoints

---
# 3.6 Exercise: Hybrid Distributed Training Strategy 


Let's configure a new NeMo GPT pretraining execution on 2 nodes (4 GPUs) with both tensor and pipeline parallel by modifying the "FIXME" in the following cell. 

To use tensor and pipeline parallel, we can keep distributed the model using in 2 dimensions: 2 GPUs for tensor parallelism and 2 GPUs for pipeline parallelism. Thus, no more resources will remain for the data parallel. In this case, the `GLOBAL_BATCH_SIZE` should be downgraded to the same size as the `MICRO_BATCH_SIZE`. This would allow us to train the model with a micro batch size of 32.

If you get stuck, view the [solution](./solutions/ex3.4.ipynb) for a hint.


In [19]:
%%writefile /dli/code/pretrain_gpt_2Node4GPU_hybrid.sh
#!/bin/bash
#SBATCH --job-name=dli_2nodes_hybrid
#SBATCH --nodes=2
#SBATCH --ntasks-per-node=2       
#SBATCH --cpus-per-task=32 ### Number of threads per task (OMP threads)
#SBATCH -o /dli/nemo/logs/%j.out
#SBATCH -e /dli/nemo/logs/%j.err

set -x -e

export OMP_NUM_THREADS=$SLURM_CPUS_PER_TASK

# Distributed training args
NNODES=2
GPUS_PER_NODE=2
TP_SIZE=2        # <--- CHANGE HERE
PP_SIZE=2         # <--- CHANGE HERE

# Distributed training 
MICRO_BATCH_SIZE=32         # <--- CHANGE HERE
GLOBAL_BATCH_SIZE=32         # <--- CHANGE HERE

# Model architecture 
NLAYERS=12
NHIDDEN=768
NHEADS=32
SEQ_LEN=1024

# Data Paths
VOCAB_FILE=/dli/data/GPT-2_assets/gpt2-vocab.json
MERGE_FILE=/dli/data/GPT-2_assets/gpt2-merges.txt
DATA_PATH=[1.0,/dli/data/GPT-2_assets/my-gpt2_text_document]

OUTPUT_PATH=/dli/nemo
LOGS_PATH=/dli/nemo/logs
NAME="2Nodes4GPUS_hybrid"       # <--- CHANGED HERE 


OPTIMIZER_ARGS=" \
            model.optim.name=fused_adam \
            model.optim.betas=[0.9,0.95] \
            model.optim.lr=6e-5 \
            model.optim.sched.min_lr=6e-6 \
            model.optim.sched.name=CosineAnnealing \
            +model.optim.sched.max_steps=800 \
            model.optim.sched.warmup_steps=80 \
            model.optim.weight_decay=1e-1 \
        "

TRAINER_ARGS=" \
            trainer.gradient_clip_val=1.0 \
            trainer.precision=32 \
            trainer.devices=$GPUS_PER_NODE \
            trainer.num_nodes=$NNODES \
            trainer.max_steps=100 \
            trainer.enable_model_summary=true \
            trainer.log_every_n_steps=10 \
            trainer.val_check_interval=20 \
            trainer.limit_val_batches=10 \
            +trainer.use_profiler=true \
        "

GPT_ARGS=" \
            model.num_layers=$NLAYERS \
            model.hidden_size=$NHIDDEN \
            model.num_attention_heads=$NHEADS \
            model.encoder_seq_length=$SEQ_LEN \
            model.data.seq_length=$SEQ_LEN \
            model.max_position_embeddings=$SEQ_LEN \
            model.micro_batch_size=$MICRO_BATCH_SIZE \
            model.global_batch_size=$GLOBAL_BATCH_SIZE \
            model.tokenizer.vocab_file=$VOCAB_FILE \
            model.tokenizer.merge_file=$MERGE_FILE \
            model.init_method_std=0.006 \
            $OPTIMIZER_ARGS \
        "

OUTPUT_ARGS=" \
            exp_manager.explicit_log_dir=$OUTPUT_PATH/$NAME \
            exp_manager.resume_if_exists=false \
            exp_manager.name=$NAME \
        "

PARALLEL_ARGS=" \
            model.tensor_model_parallel_size=$TP_SIZE \
            model.pipeline_model_parallel_size=$PP_SIZE \
        "

export CMD=" \
            python /dli/code/NeMo/examples/nlp/language_modeling/megatron_gpt_pretraining.py \
            --config-path=/dli/code/NeMo/examples/nlp/language_modeling/conf/ \
            --config-name=megatron_gpt_config.yaml \
            $TRAINER_ARGS \
            $PARALLEL_ARGS \
            $GPT_ARGS \
            $OUTPUT_ARGS \
            model.data.data_prefix=$DATA_PATH \
            model.data.data_impl=mmap \
            model.data.splits_string=\"949,50,1\" \
        "

clear; srun --jobid $SLURM_JOBID bash -c 'NCCL_DEBUG=INFO $CMD' 2>&1 | tee -a $LOGS_PATH/$NAME.txt

Overwriting /dli/code/pretrain_gpt_2Node4GPU_hybrid.sh


Now, let's submit the previous sbatch script [pretrain_gpt_2Node4GPU_hybrid.sh](./code/pretrain_gpt_2Node4GPU_hybrid.sh) for hybrid multi-node run and check the SLURM queue using the `squeue` command.

In [20]:
# Submit the 2 nodes jobs
!sbatch /dli/code/pretrain_gpt_2Node4GPU_hybrid.sh

# Check the SLURM queue
!squeue

Submitted batch job 14
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
                14  slurmpar dli_2nod     root PD       0:00      2 (None)



To understand the performance of the NeMo GPT-3 pretraining, we can check the generated logs during the execution.

Let's first look at the generated [logs](./nemo/logs/2Nodes4GPUS_hybrid.txt) and check world size of our executed hybrid run. We should see this:

```
Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/4
Initializing distributed: GLOBAL_RANK: 1, MEMBER: 2/4
Initializing distributed: GLOBAL_RANK: 2, MEMBER: 3/4
Initializing distributed: GLOBAL_RANK: 3, MEMBER: 4/4
```


In [21]:
!sleep 20
!grep "Initializing distributed:" /dli/nemo/logs/2Nodes4GPUS_hybrid.txt

Initializing distributed: GLOBAL_RANK: 1, MEMBER: 2/4
Initializing distributed: GLOBAL_RANK: 3, MEMBER: 4/4
Initializing distributed: GLOBAL_RANK: 2, MEMBER: 3/4
Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/4


Let's check the training performance and GPU0 memory allocation and deallocation of our run. You should see a graph similar to the following: 

<img src="images/profiling_hybrid.png" width="1280"/>

What is the constant time step between the forward and backward pass? Discuss with the instructor.

In [22]:
!grep Epoch /dli/nemo/logs/2Nodes4GPUS_hybrid.txt | tail

Great, before moving on, let's release some disk space by deleting the unnecessary checkpoints generated by the previous execution.


In [23]:
# Clean the checkpoints 
!rm -rf /dli/nemo/2Nodes4GPUS_hybrid/checkpoints

---
<h2 style="color:green;">Congratulations!</h2>

Before moving on, we need to make sure no jobs are still running or waiting in the queue. 

In [24]:
# Check the SLURM jobs queue 
!squeue

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
                14  slurmpar dli_2nod     root  R       0:32      2 slurmnode[1-2]


If there are still jobs running or pending, execute the following cell to cancel all the jobs using the `scancel` command. 

In [25]:
# Cancel admin user jobs
!scancel -u $USER

# Check again the SLURM jobs queue (should be either empty, or the status TS column should be CG)
!squeue

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
                14  slurmpar dli_2nod     root CG       0:36      2 slurmnode[1-2]


Next, you will see how to optimize the GPT pretraining using techniques such as Mixed Precision, Gradient Accumulation and Activation Checkpointing. Move on to [04_GPT_LM_pretrainings_optimizations.ipynb](04_GPT_LM_pretrainings_optimizations.ipynb).